In [11]:
import stackview
from skimage.io import imread, imshow
from skimage.filters import gaussian

In [3]:
image = imread('data/Haase_MRT_tfl3d1.tif', plugin='tifffile')

In [7]:
stackview.slice(image, display_width=200, display_height=200, continuous_update=True)

In [31]:
modified_image = gaussian(image, sigma=3)

In [33]:
stackview.curtain(image, modified_image * 65537, display_height=300, display_width=300, continuous_update=True)

In [35]:
stackview.orthogonal(image, display_height=250, display_width=250, continuous_update=True)